In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime 

import csv
import requests

import time
import concurrent.futures

from pairs_selection.api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap

#PCA, Clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [2]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2020, 8, 9), datetime(2022, 2, 20)]
    )

In [3]:
tker_dict = val.get_tkers(settings)

In [4]:
valid_lst = val.get_valid_tkers(tker_dict)

Excluded: FTVI, FinTech Acquisition Corp VI - Class A
Excluded: RELIW, Reliance Global Group Inc - Warrants - Series A (01/02/2026)
Excluded: UFS, Domtar Corporation
Excluded: VLAT, Valor Latitude Acquisition Corp - Class A
Excluded: DMYT-U, Rush Street Interactive Inc - Units (1 Ord Class A & 1/2 War)
Excluded: BEAT, BioTelemetry Inc
Excluded: PORT-U, Southport Acquisition Corp - Units (1 Ord Class A & 1/2 War)
Excluded: KSU-P, Kansas City Southern
Excluded: TCO-P-K, Taubman Centers Inc
Excluded: ARTEU, Artemis Strategic Investment Corp - Units (1 Ord Share Class A & 1/2 War)
Excluded: TRTL-WS, TortoiseEcofin Acquisition Corp III - Warrants (01/01/9999)
Excluded: DUAL, North Shore Dual Share Class ETF
Excluded: HIII, Hudson Executive Investment Corp III - Class A
Excluded: SVNAW, 7 Acquisition Corp - Warrants (05/11/2026)
Excluded: MSON, Misonix Inc
Excluded: HCNEU, Jaws Hurricane Acquisition Corp - Units (1 Ord Share Class A & 1/4 War)
Excluded: FSEC, Fidelity Investment Grade Securi

In [5]:
test_df = pd.DataFrame(valid_lst)
display(test_df)
test_lst = test_df[0].tolist()
#print(test_lst)

,0,1,2,3,4,5,6
0,FAAR,First Trust Alternative Absolute Return Strate...,NASDAQ,ETF,2016-05-23,null,Active
1,SVVC,Firsthand Technology Value Fund Inc,NASDAQ,Stock,2011-04-28,null,Active
2,GHL,Greenhill & Co Inc,NYSE,Stock,2004-05-06,null,Active
3,UGRO,Urban-gro Inc,NASDAQ,Stock,2019-11-14,null,Active
4,TLT,iShares 20+ Year Treasury Bond ETF,NASDAQ,ETF,2002-07-26,null,Active
...,...,...,...,...,...,...,...
8051,CMBM,Cambium Networks Corp,NASDAQ,Stock,2019-06-26,null,Active
8052,TECH,Bio-Techne Corp,NASDAQ,Stock,1992-12-09,null,Active
8053,PWC,Invesco Dynamic Market ETF,NYSE ARCA,ETF,2003-05-01,null,Active
8054,SLG-P-I,SL Green Realty Corporation Preferred Series I,NYSE,Stock,2002-07-01,null,Active


In [6]:
function = "TIME_SERIES_INTRADAY_EXTENDED"
ticker_lst = test_lst[:100]
interval = "1min"
slice = ["year1month1"] #[f"year1month{i}" for i in range(1, 4)] #first three months of intraday data
outputsize = "compact"
test_urls = wrap.intraday_url(settings, function, ticker_lst, interval, outputsize = outputsize, output = "lst")

s = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor: 
    futures = []
    data = []
    for url in test_urls: 
        futures.append(executor.submit(wrap.get_csv_data, url = url))
    for future in concurrent.futures.as_completed(futures): 
        data.append(future.result())

e = time.time()
print(f'Total time elapsed: {e-s} seconds')

s = time.time()
result2 = []
for i in test_urls: 
    result2.append(wrap.get_csv_data(i))
e = time.time()
print(f'Total time elapsed: {e-s} seconds')

Total time elapsed: 15.202145099639893 seconds
Total time elapsed: 161.84904742240906 seconds


In [7]:
print(test_urls[0])
print(len(data[4]))

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=FAAR&interval=1min&outputsize=compact&datatype=csv&apikey=MD27QTTVAK3AJBUQ
2563


# PCA

Without considering VWAP yet

In [98]:
#Maybe minute interval not v good since some stocks don't have such frequency
#Get the stock data with most number of recorded prices
for i,v in enumerate(data):
    tracker=0
    ind=0
    if len(v)>tracker:
        tracker=len(v)
    ind=i

print(ind)
stock_nm=test_lst[ind]

#Get pct_change, and transpose series such that it is 
# test_price=pd.to_numeric(test_df['close']).pct_change()
# test_price=test_price[1:].to_frame().transpose()
# test_price

99


In [101]:
#This will set the timestamps (columns) for each stock, since it is the longest
headr=pd.DataFrame(data[ind][1:],columns=data[ind][0])['time'].to_list()
